In [1]:
pip install icecream

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q tabula-py

Note: you may need to restart the kernel to use updated packages.


In [279]:
import pandas as pd
import re
from icecream import ic
import glob
import os  ## allows you to navigate, create, delete folders
from pathlib import Path ## allows to create paths to files and folders
import tabula ## Allows reading tables off pdfs

In [203]:
path1 = "pql_full_list.pdf" # Create a path to the target file 

In [277]:
## Run tabula on the pdf to bring in all the tables
pql_tables_all = tabula.read_pdf(path1, pages = "all")

## Only the even indexed dfs in the list are useful to us, so cut out the odd numbered items.
## The tables import with some weird formatting at the top and bottom. See the notes within the for loop for explainations.

pql_df_all = []
table_counter = -1

for df in pql_tables_all:
    table_counter += 1
    if (table_counter % 2) == 0:
        ## Drop the stray first two lines
        df.drop(index=df.index[:2], axis=0, inplace=True)
        ## Replace the auto imported header with the correct header line
        df.columns = df.iloc[header_row]
        ## Drop the repeated header 
        df = df.iloc[1:]
        ## Drop the NaN final row
        df = df.iloc[:-1 , :]
        ## Reset the index
        df.reset_index(drop=True, inplace=True)
        ## Remove all the instances of "\r" (carriage return)
        df = df.replace(r'\r', ' ', regex=True)
        ## Drop unncessesary columns that mess with formatting when exported as CSV files
        df.drop(['CONTACT EMAIL ADDRESS'], axis=1)
        ## Create a full address column for geocoding
        df['full_address'] = df['STREET NUMBER'] + " " + df['STREET NAME'] + " " + df['CITY'] + " " + df['STATE'] + " " + df['ZIP CODE']
        pql_df_all.append(df)
    else:
        print(f'not appending {table_counter}')

not appending 1
not appending 3
not appending 5
not appending 7


In [210]:
## We're ready to start geocoding! Import the geopy packages

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [278]:
## Create a function to geocode each dataframe in the list. 

def geocoder(df, address_column_name):
    '''
    Gives the geocode (lat, long) of an address
    df = some dataframe with a column for addresses
    address_column_name = where in the df the addresses are held
    '''
    # Nominatim geocoder for OpenStreetMap data with RateLimiter
    geocoder = RateLimiter(Nominatim(user_agent='tutorial').geocode, min_delay_seconds=1)
    df['Location'] = df[address_column_name].apply(geocoder)

    # add latitude and longitude to dataframe
    df['Latitude'] = df['Location'].apply(lambda loc: loc.latitude if loc else None)
    df['Longitude'] = df['Location'].apply(lambda loc: loc.longitude if loc else None)

geocoded_dfs = []

for df in pql_df_all:
    geocoder(df, 'full_address')
    geocoded_dfs.append(df)
    
## For QGIS, I want each table in its own CSV file, so:
## Create a function to create CSV files:

def create_csv(df_name, file_name):
    '''
    Export your dataframe as a csv
    argument 1 = your dataframe name
    argument 2 = your file name as a string. Must include .csv
    For example "my_data.csv"
    '''
    df_name.to_csv(file_name, encoding='utf-8', index=False )
    print(f"{file_name} is in your local folder!")

for index, df in enumerate(geocoded_dfs, start=1):
    create_csv(df, f"dcas_geocoded_df_{index}.csv")
    print(f'Geocoded csv file {index} is in your folder!')
    
## The files will now need to be cleaned for what geopy was unable to geocode!


RateLimiter caught an error, retrying (0/2 tries). Called with (*('1350 Broadway New York NY 10018',), **{}).
Traceback (most recent call last):
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/slabowitz/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto


dcas_geocoded_df_1.csv is in your local folder!
Geocoded csv file 1 is in your folder!
dcas_geocoded_df_2.csv is in your local folder!
Geocoded csv file 2 is in your folder!
dcas_geocoded_df_3.csv is in your local folder!
Geocoded csv file 3 is in your folder!
dcas_geocoded_df_4.csv is in your local folder!
Geocoded csv file 4 is in your folder!
